In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline



mlflow server -h 0.0.0.0 --default-artifact-root s3://mlflow-models-valval

export MLFLOW_TRACKING_URI=http://ec2-18-201-150-139.eu-west-1.compute.amazonaws.com:5000/

In [2]:
import mlflow

MLFLOW_TRACKING_URI = "http://ec2-18-201-150-139.eu-west-1.compute.amazonaws.com:5000"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-models-valval/174232467941290416', creation_time=1718551213416, experiment_id='174232467941290416', last_update_time=1718551213416, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [3]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [4]:
df_train = read_dataframe('../../01-intro/data/green_tripdata_2021-01.parquet')
# 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet'
df_val = read_dataframe('../../01-intro/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [5]:
# with mlflow.start_run():
#     params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
#     mlflow.log_params(params)

#     dv = DictVectorizer()
#     model = RandomForestRegressor(**params, n_jobs=-1)
    
#     X_train = dv.fit_transform(dict_train)
#     model.fit(X_train, y_train)

#     X_val = dv.transform(dict_val)
#     y_pred = model.predict(X_val)

#     rmse = mean_squared_error(y_pred, y_val, squared=False)
#     print(params, rmse)
#     mlflow.log_metric('rmse', rmse)

#     mlflow.sklearn.log_model(model, artifact_path="model")

#     with open('dict_vectorizer.bin', "wb") as f_out:
#         pickle.dump(dv, f_out)

#     mlflow.log_artifact('dict_vectorizer.bin')

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )
    
    pipeline.fit(dict_train, y_train)

    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [7]:
from mlflow.tracking import MlflowClient

In [ ]:
RUN_ID = '57db38f168e342cfa916bff9f7953a03'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv